In [0]:
# Import necessary libraries
from pyspark.sql import SparkSession

# Create a Spark session
spark = SparkSession.builder \
    .appName("DeltaLakeETL") \
    .getOrCreate()




In [0]:
# Create a DataFrame for customer data
customer_data = [
    (2001, 'Anu', 'India'),
    (2002, 'Meena', 'Canada'),
    (2003, 'Tanisha', 'New York')
]

customer_columns = ["customer_id", "customer_name", "location"]
customer_df = spark.createDataFrame(customer_data, customer_columns)

# Write the DataFrame to Delta format
customer_df.write.format("delta").mode("overwrite").save("dbfs:/FileStore/delta/customer_dim")


In [0]:
# Create a DataFrame for order data
order_data = [
    (1001, 1, 2001, 1, 139.99, '2023-09-27 12:30:00'),
    (1002, 2, 2002, 2, 934.99, '2023-09-27 14:45:00'),
    (1003, 3, 2003, 1, 543.99, '2023-09-27 15:30:00')
]

order_columns = ["order_id", "product_id", "customer_id", "quantity", "order_amount", "order_date"]
order_df = spark.createDataFrame(order_data, order_columns)

# Write the order DataFrame to Delta format
order_df.write.format("delta").mode("overwrite").save("dbfs:/FileStore/delta/order_fact")


In [0]:
import dlt

@dlt.table
def order_fact():
    return spark.read.format("delta").load("dbfs:/FileStore/delta/order_fact")

print("delta live table created")

delta live table created


Name,Type
order_id,bigint
product_id,bigint
customer_id,bigint
quantity,bigint
order_amount,double
order_date,string


In [0]:
import dlt

@dlt.table
def customer_dim():
    return spark.read.format("delta").load("dbfs:/FileStore/delta/customer_dim")

print("delta live table created")

delta live table created


Name,Type
customer_id,bigint
customer_name,string
location,string


In [0]:
df_dltCustomers = spark.read.format("delta").load("dbfs:/FileStore/delta/customer_dim")
df_dltCustomers.show()


+-----------+-------------+--------+
|customer_id|customer_name|location|
+-----------+-------------+--------+
|       2003|      Tanisha|New York|
|       2002|        Meena|  Canada|
|       2001|          Anu|   India|
+-----------+-------------+--------+



In [0]:
df_dlt_Order_fact = spark.read.format("delta").load("dbfs:/FileStore/delta/order_fact")
df_dlt_Order_fact.show()


+--------+----------+-----------+--------+------------+-------------------+
|order_id|product_id|customer_id|quantity|order_amount|         order_date|
+--------+----------+-----------+--------+------------+-------------------+
|    1002|         2|       2002|       2|      934.99|2023-09-27 14:45:00|
|    1003|         3|       2003|       1|      543.99|2023-09-27 15:30:00|
|    1001|         1|       2001|       1|      139.99|2023-09-27 12:30:00|
+--------+----------+-----------+--------+------------+-------------------+

